In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
from matplotlib import pyplot
from matplotlib.pyplot import plot
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity


data_=pd.read_csv('comparative_emotion_analyzer.csv')
#data_
data=data_.drop(['Unnamed: 0','Song_ID','Artists','Artists_Id','Lengthofsing_ms','Modemajor1_happier','Release_date','Album_type','source','Popularity'],axis=1)
data['Mode']=data_['Modemajor1_happier']
for i in range(1,data.shape[1]):
    data.iloc[:,i]=data.iloc[:,i].apply(lambda x:round(x,3))
#data

n = int(input('Enter how many most frequent songs users listened in past month! '))
m = int(input('How many steps closer to your mood? '))
z = int(input('How many closest recommendations, you think we should offer?'))


# def song_recommendation(data,n):
#     data=data.iloc[0:n,:].reset_index().drop('index',axis=1)
    
def two_least_similar_choice(data):
    #print(data)
    cosim=cosine_similarity(data.iloc[:,1:19],data.iloc[:,1:19])
    j=1
    res2=list()
    min_=cosim.min()
    for i in range(len(cosim)):
        #print('i',i)
        for k in range(j):
            #print('j',j)
            if cosim[i][k]==min_:
                res2.extend([i,j])
                print(res2)
                #print(res)
        #print(i)

        j+=1  
        #print(j)
    a=list(set(res2))
    print(a)
    print(data)
    song1=data['Song'][a[0]-1]
    song2=data['Song'][a[1]-1]
    


    #Have create buttons here and take the input
    print(f"Chose among songs:   {song1}   &   {song2} .") 
    selection=input('Which one? ')
    return selection

# def most_similar_songset(data,song):
#     n=data.shape[0]
    
def similarity(data):
    return cosine_similarity(data.iloc[:,1:19],data.iloc[:,1:19])

def recursive(data,z):
    print('recursive entry',data.shape)
    k=input(f'{data.shape[0]} songs? : Y or N')
    if k=='Y' or k=='y':
        print(data['Song'])
    if data.shape[0]<z:
        print (data['Song'])
    else:
        cosim =similarity(data)
        sel=two_least_similar_choice(data)
        ind=data.set_index('Song').index.get_loc(sel)
        a=list(cosim[ind])
        list1=sorted(range(len(a)), key=lambda i: a[i], reverse=True)[0:len(a)//2]#BECAUSE FIRST IS THE SONG ITSELF WITH COSIN_SIM==1
        #list1
        final_list=list()
        for i in list1:
            final_list.append(data.iloc[i,:].tolist())
        data_final=pd.DataFrame(final_list,columns=data.columns)
        recursive(data_final,z)
        

def final_model(data,n,m,z):
    model_data=data.iloc[0:25*n,:]
    model_data_norm= StandardScaler().fit_transform(model_data.iloc[:,1:])
    df1=model_data['Song']
    df1.reset_index(drop=True,inplace=True)
    df2=pd.DataFrame(model_data_norm[:,:],columns=list(model_data.columns)[1:])
    df2.reset_index(drop=True,inplace=True)
    data_final=pd.concat([df1,df2],axis=1)
    
    for i in range(1,data_final.shape[1]):
        data_final.iloc[:,i]=data_final.iloc[:,i].apply(lambda x:round(x,3))
    data_final=data_final.iloc[0:n,:].reset_index().drop('index',axis=1)
    
    #stripping the spaces in song names
    data_final['Song']=data_final['Song'].apply(lambda x: x.strip())
    k=z
    print(data_final.shape)
    recursive(data_final,k)
    #Choosing top n songs
final_model(data,n,m,z)